In [ ]:
import pandas as pd
import numpy as np
import re
import requests
import xmltodict
import time
import pickle

In [ ]:
dataset = pd.read_csv("assylum_cases.csv")

In [ ]:
#Add year column to the dataframe
dataset['year'] = dataset.apply(lambda row: (re.search("\d\d\d\d", row.case))[0], axis=1)

In [ ]:
def get_text_zaaknummer_ecli_given_list(ECLI_list):
    
    DETAILED_LINK_URL = "https://data.rechtspraak.nl/uitspraken/content?id={}"
    detailed_link_list = [ecli for ecli in ECLI_list]

    
    for detailed_link in detailed_link_list:
        ecli_temp = get_ecli_from_detailed_link(detailed_link)
        detailed_sentence = requests.get(
            DETAILED_LINK_URL.format(ecli_temp))
        
    return detailed_sentence
    


def get_ecli_from_detailed_link(detailed_link):
    try:
        return detailed_link.split("=")[-1]
    except:
        return None
    
def get_zaaknummer_from_called_api(called_api_page):
    doc = xmltodict.parse(called_api_page.text)
    return \
        doc["open-rechtspraak"]["rdf:RDF"]["rdf:Description"][0]["psi:zaaknummer"]["#text"]    



In [ ]:
#Extract cases where titles are labeled

structured_cases = []

for i in range(len(ECLI_list) - 1): 
    print (i, end="\r")
    detailed_sentence = get_text_zaaknummer_ecli_given_list(ECLI_list[i:i+1])
    if '<title>' in detailed_sentence.text:
        structured_cases.append(i)    
    time.sleep(0.1)
        

In [ ]:
# with open("structured.txt", "wb") as fp:   #Pickling
#     pickle.dump(structured_cases, fp)

In [ ]:
with open("structured.txt", "rb") as fp:   # Unpickling
    structured_case_ids = pickle.load(fp)

In [ ]:
len(structured_case_ids)

In [ ]:
structured_cases_dataset = dataset
structured_cases_dataset['structured'] = np.NAN


In [ ]:
for index, row in structured_cases_dataset.iterrows():
    if index in structured_case_ids:
        structured_cases_dataset.at[index,'structured'] = 1
    else:
        structured_cases_dataset.at[index,'structured'] = 0

In [ ]:
structured_cases_dataset.to_csv("asylum_cases_structure.csv")